In [1]:
import numpy as np


# To read the training data and make a vocabulary and dictiornary to index the chars
class DataReader:
    def __init__(self, path, seq_length, data="", progress_len=5, max_trace_len = 200):
        #uncomment below , if you dont want to use any file for text reading and comment next 2 lines
        self.data = "Today we will be talking about RNN and LSTM. RNN stands for recurrent neural network, " + \
                    "the special property of RNN is its ability to use the results from the past to generate a new result. " + \
                    "LSTM stands for Long-Short Term Memory, " + \
                    "LSTM is a form of RNN but differs from traditional RNN by having another value representing the long-term memory, " + \
                    "and a more complex algorithm to handle the long/short-term memories. LSTM is better at handling long-term meories and can also resolve exploding/vanishing gradients."
        if len(data) != 0:
            self.data = data
        #self.fp = open(path, "r")
        #self.data = self.fp.read()
        #find unique chars
        chars = list(set(self.data))
        #create dictionary mapping for each char
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)}
        #total data
        self.data_size = len(self.data)
        print(chars)
        #num of unique chars
        self.vocab_size = len(chars)
        self.pointer = 0
        self.seq_length = seq_length
        self.progress_len = progress_len
        self.max_trace_len = max_trace_len

    def next_batch(self):
        input_start = self.pointer
        input_end = self.pointer + self.seq_length
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        self.pointer += self.progress_len
        if self.pointer + self.seq_length + 1 >= self.data_size:
            # reset pointer, start new epoch
            self.pointer = -1
        trace_start = 0
        if self.pointer > self.max_trace_len:
            trace_start = self.pointer - self.max_trace_len
        prev_input = [self.char_to_ix[ch] for ch in self.data[trace_start:input_start]]
        return prev_input, inputs, targets

    def new_epoch(self):
        return self.pointer == -1
    def start_epoch(self):
        self.pointer = 0

    def close(self):
        self.fp.close()

In [2]:
  
class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        # model parameters
        self.U = np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))
        self.V = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        self.b = np.zeros((hidden_size, 1)) # bias for hidden layer
        self.c = np.zeros((vocab_size, 1)) # bias for output
        
        # memory vars for adagrad, 
        #ignore if you implement another approach
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)

    def softmax(self, x):
        p = np.exp(x- np.max(x))
        return p / np.sum(p)
        
    def forward(self, start, inputs):
        xs, hs, os, ycap = {}, {}, {}, {}

        oh_start = np.zeros((self.vocab_size, 1))
        hs_start = np.zeros((self.hidden_size, 1))
        for c in start:
            oh_start[c] = 1
            hs_start = np.tanh(np.dot(self.U, oh_start) + np.dot(self.W, hs_start)+self.b)
            oh_start[c] = 0
        
            
        hs[-1] = hs_start

        
        for t in range(len(inputs)):
            xs[t] = np.zeros((self.vocab_size,1))
            xs[t][inputs[t]] = 1 # one hot encoding , 1-of-k
            hs[t] = np.tanh(np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b) # hidden state
            os[t] = np.dot(self.V,hs[t]) + self.c # unnormalised log probs for next char
            ycap[t] = self.softmax(os[t]) # probs for next char
        return xs, hs, ycap
        
    def backward(self, xs, hs, ps, targets):
            # backward pass: compute gradients going backwards
            dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
            db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
            dhnext = np.zeros_like(hs[0])
            for t in reversed(range(self.seq_length)):
                dy = np.copy(ps[t])
                #through softmax
                dy[targets[t]] -= 1 # backprop into y
                #calculate dV, dc
                dV += np.dot(dy, hs[t].T)
                dc += dc
                #dh includes gradient from two sides, next cell and current output
                dh = np.dot(self.V.T, dy) + dhnext # backprop into h
                # backprop through tanh non-linearity 
                dhrec = (1 - hs[t] * hs[t]) * dh  #dhrec is the term used in many equations
                db += dhrec
                #calculate dU and dW
                dU += np.dot(dhrec, xs[t].T)
                dW += np.dot(dhrec, hs[t-1].T)
                #pass the gradient from next cell to the next iteration.
                dhnext = np.dot(self.W.T, dhrec)
            # clip to mitigate exploding gradients
            for dparam in [dU, dW, dV, db, dc]:
                np.clip(dparam, -5, 5, out=dparam) 
            return dU, dW, dV, db, dc
    
    def loss(self, ps, targets):
        """loss for a sequence"""
        # calculate cross-entrpy loss
        return sum(-np.log(ps[t][targets[t],0]) for t in range(self.seq_length))
        
    
    def update_model(self, dU, dW, dV, db, dc):
        # parameter update with adagrad
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8) # adagrad update
                
                
    def sample(self, h, seed_ix, n):
            """
            sample a sequence of integers from the model
            h is memory state, seed_ix is seed letter from the first time step
            """
            x = np.zeros((self.vocab_size, 1))
            x[seed_ix] = 1
            ixes = []
            for t in range(n):
                h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
                y = np.dot(self.V, h) + self.c
                p = np.exp(y)/np.sum(np.exp(y))
                ix = np.random.choice(range(self.vocab_size), p = p.ravel())
                x = np.zeros((self.vocab_size,1))
                x[ix] = 1
                ixes.append(ix)
            return ixes

    def train(self, data_reader, threshold = 0.01, max_epoch = 100, print_string = ""):
        epoches = 0
        loss = threshold + 1
            
        while (loss > threshold and max_epoch > epoches):
            data_reader.start_epoch()
            exec(print_string)
            while not data_reader.new_epoch():
                start, inputs, targets = data_reader.next_batch()
                for i in range(0, len(start), 15):
                    xs, hs, ps = self.forward(start[i:], inputs)
                    dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                    self.update_model(dU, dW, dV, db, dc)

                # xs, hs, ps = self.forward(start, inputs)
                # dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                # self.update_model(dU, dW, dV, db, dc)
                
            epoches += 1            
            loss = self.loss(ps, targets)
            print(f"Finished {epoches} epoches, Loss: {loss}")

    def predict(self, data_reader, start, n):

        #initialize input vector
        x = np.zeros((self.vocab_size, 1))
        chars = [ch for ch in start]
        ixes = []        
        h = np.zeros((self.hidden_size,1))

        for i in range(len(chars) - 1):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            ixes.append(ix)
            x[ix] = 0

        ix = data_reader.char_to_ix[chars[-1]]
        x[ix] = 1
        ixes.append(ix)
        # predict next n chars
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            x[ix] = 0
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt

In [3]:

seq_length = 25
#read text from the "input.txt" file
data_reader = DataReader("input.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader, \
         print_string = """
print(f"starting epoch: {epoches}")
predicted = self.predict(data_reader, 'memory ', 50)
print(f'Prediction 1 (memory): {predicted}')
            
predicted = self.predict(data_reader, 'LSTM ', 50)
print(f'Prediction 2 (LSTM): {predicted}')
            
predicted = self.predict(data_reader, 'complex ', 50)
print(f'Prediction 3 (complex): {predicted}') """)


predicted = rnn.predict(data_reader, 'memory ', 50)
print(f"Prediction 1 (memory): {predicted}")

predicted = rnn.predict(data_reader, 'LSTM ', 50)
print(f"Prediction 2 (LSTM): {predicted}")

predicted = rnn.predict(data_reader, 'complex ', 50)
print(f"Prediction 3 (complex): {predicted}")


['d', 'a', 'i', 'N', 's', 'L', 'b', 'u', 'w', '/', 'T', 'o', 'M', ' ', 'f', 'x', 'y', 'c', 'k', 'n', 't', 'S', 'h', ',', 'R', '.', 'l', '-', 'p', 'v', 'm', 'e', 'g', 'r']
starting epoch: 0
Prediction 1 (memory): memory LfscLnTRo -nrklktkR-uc -ttLc-Mff-bex,savtcTkduf.si
Prediction 2 (LSTM): LSTM aeLhfc,ldTbernciRko doRahmv/M/xfi,dTvmTysolkkns.vm
Prediction 3 (complex): complex .lsaoRd/ovSs.uN.hnh-wyoLplprrLMR.dN,e-rN,uTRmaegfi
Finished 1 epoches, Loss: 23.181388986584807
starting epoch: 1
Prediction 1 (memory): memory he vaninrxnisg-vlcding/viadoiandsgandaorieie explo
Prediction 2 (LSTM): LSTM inN ang ggkms andshind grtdieng-vandsging-annusesn
Prediction 3 (complex): complex ani ga/g.esong gtiha taidatand ing ing stinisg/vin
Finished 2 epoches, Loss: 2.185397328809108
starting epoch: 2
Prediction 1 (memory): memory hestand candshandseng ang andisndshindsorisslvang 
Prediction 2 (LSTM): LSTM ing gandam ong-gexploding/vonishing gradie handsha
Prediction 3 (complex): complex and hangstsong

In [4]:
predicted = rnn.predict(data_reader, 'LSTM is bet', 50)
print(f"Prediction Special: {predicted}")
predicted = rnn.predict(data_reader, 'Today, we wi', 50)
print(f"Prediction Special: {predicted}")


Prediction Special: LSTM is betttretorm memories. LSTM is better at handling long
Prediction Special: Today, we winf as les and femory, LSTM is a form of RNN but di


Train model with even easier input. 

In [5]:

seq_length = 25
#read text from the "input.txt" file
data_reader = DataReader("input.txt", seq_length, \
                         data="This is a even simpler string, hopefully the model can now perform well with predicting the outputs.")
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader)


predicted = rnn.predict(data_reader, 'This ', 50)
print(f"Prediction 1 (This): {predicted}")

predicted = rnn.predict(data_reader, 'simpler ', 50)
print(f"Prediction 2 (simpler): {predicted}")

predicted = rnn.predict(data_reader, 'perform ', 50)
print(f"Prediction 3 (perform): {predicted}")


['d', 's', 'i', 'a', 'u', 'w', 'T', 'o', ' ', 'f', 'y', 'c', 't', 'n', 'h', ',', '.', 'l', 'p', 'v', 'm', 'e', 'r', 'g']
Finished 1 epoches, Loss: 85.68324409551748
Finished 2 epoches, Loss: 83.45195208973722
Finished 3 epoches, Loss: 77.18536981675688
Finished 4 epoches, Loss: 86.80730814939801
Finished 5 epoches, Loss: 67.92017099665605
Finished 6 epoches, Loss: 64.25119187597937
Finished 7 epoches, Loss: 59.8476005697024
Finished 8 epoches, Loss: 49.96565369944913
Finished 9 epoches, Loss: 46.723429474771756
Finished 10 epoches, Loss: 38.43270715726767
Finished 11 epoches, Loss: 32.70403745485998
Finished 12 epoches, Loss: 30.2225219600895
Finished 13 epoches, Loss: 22.574215652608267
Finished 14 epoches, Loss: 20.329430947849424
Finished 15 epoches, Loss: 19.20493144012723
Finished 16 epoches, Loss: 15.937614534050809
Finished 17 epoches, Loss: 15.93332180764688
Finished 18 epoches, Loss: 12.269067183504685
Finished 19 epoches, Loss: 10.926656360234047
Finished 20 epoches, Loss: 9.